#  trail run estimate

at the trail, it is especially important to correctly asses duration of the activitity, since it is a safety factor.

the goal here is to asses correctly the trail run time, point out potentially too steep regions, snow conditions, need for light, view points

time of walk and of run

In [ ]:
android app

faster run or walk

In [ ]:
fractions=[0,0.5, 1]
i=0

tuple((int, 255*np.arrcm.jet(i/len(fractions))[:-1]))

# model

power modes: fixed VAM (power to potential), fixed speed (power to running frix)

# TODO

find walk regions by cadence
run and walk by grade, metrc

In [ ]:
#logging.basicConfig(level=logging.DEBUG)

from importlib import reload  # Not needed in Python 2
import logging
reload(logging)

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [ ]:
%matplotlib notebook
import matplotlib as mpl
from matplotlib import pylab as plt
from matplotlib import cm as cm
import numpy as np
import requests
import time
from IPython.display import display, HTML
from urllib.parse import urlencode
import yaml

In [ ]:
c = yaml.safe_load(open("strava-client.yaml"))

client_id = c['client_id']
client_secret = c['client_secret']

In [ ]:
# auth?

print("http://www.strava.com/oauth/authorize?" + urlencode(dict(
        client_id=client_id,
        response_type="code",
        redirect_uri="http://localhost/exchange_token",
        #redirect_uri="http://trail.volodymyrsavchenko.com:8000/exchange_token",
        approval_prompt="force",
        scope="activity:read_all",
    )))

code=input()

r=requests.post("https://www.strava.com/api/v3/oauth/token",
  data=dict(
      client_id=client_id,
      client_secret=client_secret,
      code=code,
      grant_type="authorization_code",
  )
).json()

token=r['access_token']

In [ ]:
token=r['access_token']

requests.get("https://www.strava.com/api/v3/athlete", headers={"Authorization": f"Bearer {token}"}).json()

In [ ]:
import time
import bravado
import bravado.client
import bravado.requests_client
import bravado.http_client

reload(bravado), reload(bravado.client), reload(bravado.requests_client), reload(bravado.http_client)

from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient

http_client = RequestsClient()
http_client.set_api_key(
    'www.strava.com', 'Bearer '+token,
    param_name='Authorization', param_in='header'
)

client = SwaggerClient.from_url(
    'https://developers.strava.com/swagger/swagger.json',
    http_client=http_client,
    config={
        'validate_swagger_spec': False,
        'validate_responses': False,
        'validate_requests': False,
    },
)
athlete = client.Athletes.getLoggedInAthlete().response().result
athlete

In [ ]:
routes = requests.get(f"https://www.strava.com/api/v3/athletes/{athlete['id']}/routes", 
                              params=dict(
                                  per_page=100
                              ),
                              headers={'Authorization': 'Bearer '+token}
                             ).json()

In [ ]:
import io
import gpxpy

for route in routes:
    print("getting route gpx for", route['id'])
    route['route_gpx'] = gpxpy.parse(io.BytesIO(requests.get(f"https://www.strava.com/api/v3//routes/{route['id']}/export_gpx", 
                                  params=dict(
                                      per_page=100
                                  ),
                                  headers={'Authorization': 'Bearer '+token}
                                 ).content))
    route['route_points'] = route['route_gpx'].tracks[0].segments[0].points

    print("got", route['route_gpx'].tracks[0])

In [ ]:
for route in routes:
    #if '21' not in route['route_gpx'].name: continue
    #if 'par KV' not in route['route_gpx'].name: continue
    
    print(f"\033[31m{route['route_gpx'].name}\033[0m", route['type'], route['sub_type'], route['distance']/1000.)


In [ ]:
route['distance'], route['elevation_gain']

In [ ]:


d_N = 2
N = 3

for route in routes:
    #if '21' not in route['route_gpx'].name: continue
    #if 'KV' not in route['route_gpx'].name: continue
    if 'MT' not in route['route_gpx'].name: continue
    
    route_points = route['route_points']

    d_route_d3 = np.array([0] + [a.distance_3d(b) for a,b in zip(route_points[:-1], route_points[1:])])
    route_d3 = np.cumsum(d_route_d3)

    route_lat = np.array([p.latitude for p in route_points])
    route_lon = np.array([p.longitude for p in route_points])
    route_altitude = np.array([p.elevation for p in route_points])
    route_grade = d_(route_altitude)/d_(route_d3, 10)*100.

    plt.figure()
    plt.title(route['route_gpx'].name)
    print(f"\033[31m{route['route_gpx'].name}\033[0m", route['type'], route['sub_type'])

    plt.scatter(
        route_lon,
        route_lat,
        c=cm.jet(route_grade/50.),
    )

    plt.grid()
    #plt.ylim([-50, 50])

    total_time_estim = 0
    i = 0
    d_time_estims = []
    for d_d, d_a in zip(d_route_d3, d_(route_altitude, 2)):
        d_time_estims.append(0)
        
        i += 1
        if i<d_N +1: continue
        if i>len(route_d3)-d_N -1: continue

        grade = d_a/d_d*100.

        estim_speed_ms = (speed_estim_for_grade(grade, lut_merged)/3600.*1000)
        d_time_estim = d_d/(estim_speed_ms)/(d_N-1)

        if not np.isnan(d_time_estim) and not np.isinf(d_time_estim):
            total_time_estim += d_time_estim
            #sum_time += d_t/(d_N-1)
            d_time_estims[-1] = d_time_estim

        #print("since", _t - ttime[0], "estim", total_time_estim, "sum", sum_time)
        #print(f"dd {d_d:.2f} {d_a:.2f} grade {grade:.2f} estim speed {estim_speed_ms*3.6:.2f}, dtime {d_time_estim:.3f} total time {total_time_estim/3600.:.2f}")

    d_time_estims=np.array(d_time_estims)
    
    print(f"total distance {np.sum(d_route_d3)/1000.:.2f} km (strava {route['distance']/1000.:.2f})")
    
    asc = d_(route_altitude, 2)
    
    print(f"cumulative elevation gain {np.sum(np.sum(d_(route_altitude, N)/(N-1))):.2f} m (strava {route['elevation_gain']:.2f})")
    print(f"total elevation gain {np.sum(asc[asc>0]):.2f} m (strava {route['elevation_gain']:.2f})")
    print(f"total time {total_time_estim/3600.:.2f} strava estim {route['estimated_moving_time']/3600.:.2f}")
    m_route_run_steep = route_grade<-20.
    m_route_run_flat = (route_grade<8.) & (route_grade>-20.)
    m_route_walk = route_grade>8.
    m_route_steep = route_grade>30.

    for n, mx in [
        ("down steep", m_route_run_steep),
        ("run flat", m_route_run_flat),
        ("walk up", m_route_walk),
        ("steep up", m_route_steep),
    ]:        
        print(f"{n:10s} "+
              f"\033[032m{np.sum(d_time_estims[mx])/np.sum(d_time_estims)*100:5.1f}%\033[0m time " +
              f"{np.sum(d_time_estims[mx])/3600.:.2f} hr " +
              f"{np.sum(d_route_d3[mx])/1000.:.2f} km",
              f"{np.sum(d_(route_altitude, N)[mx])/(N-1):.2f} VAM",
             )
        
        
    if False:
        plt.figure()
        plt.plot(
            np.cumsum(d_route_d3),
            route_altitude,
        )
        plt.plot(
            np.cumsum(d_route_d3),
            np.cumsum(d_(route_altitude, N))/(N-1),
        )
        plt.grid()
        
    #break
    
    
    


In [ ]:
def d_(x, n=10):
    d_filter = np.zeros(n)
    d_filter[0] = 1
    d_filter[-1] = -1
    R = np.convolve(x, d_filter, mode='same')
    
    R[:1]=0
    R[-1:]=0
    
    return R

d_(route_altitude, N)

In [ ]:
if False:
    activities = client.Activities.getLoggedInAthleteActivities(
        after=int(time.time() - 720*24*3600),
        perPage=100,
    ).response().result
else:
    activities = requests.get("https://www.strava.com/api/v3/athlete/activities", 
                              params=dict(
                                  per_page=100
                              ),
                              headers={'Authorization': 'Bearer '+token}
                             ).json()

In [ ]:
for activity in activities:
    if 'Ride' in activity['name'] or activity['total_elevation_gain'] < 300:
        activity.pop('streams', None)
        continue
        
        
    
    activity['streams'] = client.Streams.getActivityStreams(
        id=activity['id'], 
        keys="time, distance, latlng, altitude, velocity_smooth, heartrate, cadence, watts, temp, moving, grade_smooth".split(", "),
        key_by_type=True,
    ).response().result


    
    print(activity['name'])

In [ ]:
def d_(x, n=10):
    d_filter = np.zeros(n)
    d_filter[0] = 1
    d_filter[-1] = -1
    return np.convolve(x, d_filter, mode='same')

In [ ]:
#plt.figure()

d_N = 10

luts = []

for activity in activities:
    if 'streams' not in activity: continue
        
    if 'Lake Winter' not in activity['name']: continue
        
    streams = activity['streams']
    
    
    distance = np.array(streams['distance']['data'])
    altitude = np.array(streams['altitude']['data'])
    ttime = np.array(streams['time']['data'])
    latlng = np.array(streams['latlng']['data'])
    cadence = np.array(streams['cadence']['data'])*2.
    heartrate = np.array(streams['heartrate']['data'])
  
    
    d_distance1 = d_(distance, 1)
    d_distance = d_(distance, d_N)
    d_altitude = d_(altitude, d_N)
    d_d3 = (d_distance**2 + d_altitude**2)**0.5
    d_time = d_(ttime, d_N)
    
    speed_kph = d_d3/d_time *3600./1000.
    vam = d_altitude/d_time    
    grade = d_altitude/d_d3*100.
    

    def dealias(x):
        mn = np.round(np.abs(x[x>0]).min(), 5)
        print("found step", mn)
        x += np.random.rand(altitude.shape[0])*mn - mn/2.
        
    dealias(cadence)
    dealias(d_altitude)
    dealias(d_distance)

    speed_kph = d_d3/d_time *3600./1000.
    vam = d_altitude/d_time*1000
    grade = d_altitude/d_d3*100.

    m = np.abs(d_altitude)<1000
    m &= np.abs(vam)<2000

    def p2d(x,y, x_bins, y_bins, mode='hist'):

        if mode == "scatter":
            plt.scatter(x,y,s=5)
        elif mode == 'hist':
            return plt.hist2d(x,y, bins=(
                    x_bins,
                    y_bins,
                )
            )
        elif mode == 'contour':
            h2, a,b  = np.histogram2d(x,y, bins=(
                    x_bins,
                    y_bins,
                )
            )

            plt.contourf(
                a[:-1],
                b[:-1],
                np.transpose(h2),
                #levels=np.logspace(np.log10(h2.max()/1000.), np.log10(h2.max()), 100)
                levels=np.linspace(0, h2.max(), 100)
            )


    for mode in 'hist',: #'contour':
        f, (ax1,ax2)= plt.subplots(1,2,figsize=(9,5))
        lut_speed_grade = p2d(
            speed_kph[m],
            #d_altitude[m]/d_time[m]*3600.,
            #60./(d_distance[m]/d_time[m]*3600./1000.),
            #d_distance[m]/d_time[m]*3600./1000.,
            #d_altitude[m]/d_time[m]*3600.,
            #d_altitude[m]/d_time[m]*3600,
            grade[m],
            mode = mode,
            x_bins = np.linspace(0, 15, 70),
            y_bins = np.linspace(-60, 60, 70),
            #y_bins = np.linspace(-2000, 2000, 70),
        )
        
        luts.append(lut_speed_grade)
        
        plt.subplots_adjust(wspace=0)
        
        ax1.scatter(latlng[:,1], latlng[:,0], 
            #c=cm.jet(np.array(streams['velocity_smooth']['data'])/6.),
            c=cm.jet((
                np.array(d_altitude*3600)/(1000.)
                #np.array(streams['altitude']['data'])-320)/(600-320)
            ),
        ))
        
        plt.title(activity['name'])
        
    print("total time", (ttime[-1] - ttime[0])/3600., "hr")
    
    x_grade = np.linspace(-50,50)
    ax2.plot(
        [speed_estim_for_grade(x, lut_merged, (0, 1.)) for x in x_grade],
        x_grade,
        c="k",
        lw=3,
    )
    ax2.plot(
        [speed_estim_for_grade(x, lut_merged, (0.3, 0.7)) for x in x_grade],
        x_grade,
        c="r",
        lw=3,
        ls=":"
    )
    
    plt.figure()
    plt.hist2d(
        grade[m], 
        heartrate[m],
        bins=(100, 100),
    )
    plt.axvline(8.)
    

    
    total_time_estim = 0
    sum_time = 0
    i = 0
    for d_d, d_a, d_t, _t in zip(d_d3, d_altitude, d_time, ttime):
        i += 1
        if i<30: continue
        if i>len(d_time)-100: continue
        
        #print(d_d, d_a)
        
        grade = d_a/d_d*100.
        
        d_time_estim = d_d/(speed_estim_for_grade(grade, lut_merged)/3600.*1000)/(d_N-1)
        
        if not np.isnan(d_time_estim)and  not np.isinf(d_time_estim):
            total_time_estim += d_time_estim
            sum_time += d_t/(d_N-1)
                        
        #print("since", _t - ttime[0], "estim", total_time_estim, "sum", sum_time)
        #print("grade", grade, "speed", d_d/d_t*3.6, "estim speed", speed_estim_for_grade(grade, lut_speed_grade), "time estim", d_time_estim, "time spent", d_t)
        
        
        
    print("total time estimate", total_time_estim/3600., "hr", "summed time", sum_time/3600., "hr")
    
    print("total run fraction", np.sum(m_run)/m_run.shape[0])
        
    #break

    #ax=plt.gca()
    #ax2=plt.twiny()
    #ax2.set_xlim(60./np.array(ax.get_xlim()))

    #plt.xlabel("pace")
    #plt.ylabel("grade")
    #plt.ylabel("VAM")

In [ ]:
np.save(open("lut_merged.npy", "wb"), lut_merged)

In [ ]:
plt.figure()
plt.hist(np.array(cadence)*2, 100)

In [ ]:
lut_merged = np.sum([l[0] for l in luts], 0), luts[0][1], luts[0][2]

In [ ]:

def speed_estim_for_grade(grade, lut_speed_grade, peak=(0.1, 0.9), plot=False):

    i_grade = np.abs(grade - lut_speed_grade[2][:-1]).argmin()

    dp = np.copy(lut_speed_grade[0][:,i_grade])
    
    cdp = np.cumsum(dp/np.sum(dp))
    
    #print(cdp)
    
    if plot:
        plt.figure()
        plt.plot(
            lut_speed_grade[1][:-1],
            cdp
        )
        plt.plot(
            lut_speed_grade[1][:-1],
            dp/np.sum(dp)*10
        )
        
        plt.axvline(np.sum(lut_speed_grade[1][:-1]*dp)/np.sum(dp))
    
    b1 = np.argmin(np.abs(cdp-peak[0]))
    b2 = np.argmin(np.abs(cdp-peak[1]))
    
    dp[:b1] = 0
    dp[b2:] = 0
    
    if plot:
        plt.plot(
            lut_speed_grade[1][:-1],
            dp/np.sum(dp)*10
        )
        
        plt.axvline(np.sum(lut_speed_grade[1][:-1]*dp)/np.sum(dp))
    

    return np.sum(lut_speed_grade[1][:-1]*dp)/np.sum(dp)

    #lut_speed_grade[2][i_grade], grade, lut_speed_grade[0][i_grade]
    
speed_estim_for_grade(-5, lut_merged, (0.2, 0.8), plot=True)

In [ ]:
np.median([0])

In [ ]:
plt.figure()

plt.hist(d_altitude[m]/d_time[m]*3600, 300)